In [ ]:

!pip install selenium
!pip install webdriver_manager

In [ ]:
# Importing Libraries
# We will be using selenium to automate to navigate to the said page. 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests #Helps in obtaining the requests from the 
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import time
import shutil
import os
import requests,json, re

In [ ]:
# Init of Selenium - Chrome WebDriver

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
#Depreciation warning can be ignored, as we are trying to primarily complete the task within the deadline :)

In [ ]:
BASE_URL="https://indianbirdsong.org/analytics/species/highest-lowest-recordings"
driver.get(BASE_URL)

### Getting the main URL's

In [ ]:
tot_url_elements=[]
for i in range(1,1000):
    url_elements = driver.find_elements("xpath","/html/body/div/div/div/div/div/div/div[2]/div/div[2]/table/tbody/tr["+str(i)+"]/th/a[contains(@href, '/birds/')]")
    if str(url_elements)=="[]":
        break
    tot_url_elements.append(url_elements[0].get_attribute("href"))

In [ ]:
# tot_url_elements

### Looping through the birds to get the audio URL

In [ ]:
audio_url_elements=pd.DataFrame(columns=["Bird Name","J found","Sound URL"])
test=0
iteri=0
for i in tot_url_elements:
    print("*******************************")
    print()
    print("Loading URL: ",i)
    driver.get(i)
    #Increase timer if Index Error occurs
    time.sleep(5)
    print("Loading done!!!!")
    bird_element = driver.find_elements("xpath","/html/body/div/div/div/div/div/div/div[3]/div[2]/div[1]/h1")
    bird_name=bird_element[0].text
    count_element = driver.find_elements("xpath","/html/body/div/div/div/div/div/div/div[3]/div[2]/div[1]/a")
    count_val=0
    try:
        count_val=int(count_element[0].text)
    except:
        print("Number fetch failed")
        count_val=600
    print("Bird Name: ",bird_name)
    print("Total Noted Entries: ",count_val)
    localfetch=0
    for j in range(1,count_val+50):
        url_elements = driver.find_elements("xpath","/html/body/div/div/div/div/div/div/div[3]/div[4]/div/div["+str(j)+"]/div[4]/div[2]/a")
        if str(url_elements)!="[]":
            audio_url_elements.at[iteri,'J found']=j
            audio_url_elements.at[iteri,'Bird Name']=bird_name
            audio_url_elements.at[iteri,'Sound URL']=url_elements[0].get_attribute("href")
            iteri+=1
            localfetch+=1
    print("-------------------------------------------------------")
    print("Fetched URLs for {} : {}".format(bird_name,localfetch))
    print("Failed / Not Available URLs for {} : {}".format(bird_name,count_val-localfetch))
    print("-------------------------------------------------------")
    print()
#             audio_url_elements.append(url_elements[0].get_attribute("href"))
#     test+=1
#     if test >=1:
#         break

### Run this for saving the Audio files 

In [ ]:
for index, row in audio_url_elements.iterrows():
#     print(row["Thumbnail URL"])
    
    if not os.path.exists(row["Bird Name"]):
        os.makedirs(row["Bird Name"])
    
    image_url=row["Sound URL"]
    filename=row["Bird Name"]+"\\"+row["Bird Name"]+" - "+str(index)+" .mp3"
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

        print('Audio sucessfully Downloaded: Location - ',filename)
    else:
        print('Audio Couldn\'t be retreived')

In [ ]:
#Nothing, dummy

# audio_url_elements['J found']=audio_url_elements['J found'].astype('int')
# audio_url_elements.loc[audio_url_elements["J found"].idxmax()]